In [1]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
Nhanes_ds = pd.read_csv("datasets/Nhanes.csv")
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
Nhanes_ds = Nhanes_ds[vars].dropna()
Nhanes_ds["Group"]  = 10 * Nhanes_ds.SDMVSTRA + Nhanes_ds.SDMVPSU
Nhanes_ds.head(10)

,BPXSY1,RIDAGEYR,RIAGENDR,RIDRETH1,DMDEDUC2,BMXBMI,SMQ020,SDMVSTRA,SDMVPSU,Group
0,128.0,62,1,3,5.0,27.8,1,125,1,1251
1,146.0,53,1,3,3.0,30.8,1,125,1,1251
2,138.0,78,1,3,3.0,28.8,1,131,1,1311
3,132.0,56,2,3,5.0,42.4,2,131,1,1311
4,100.0,42,2,4,4.0,20.3,2,126,2,1262
5,116.0,72,2,1,2.0,28.6,2,128,1,1281
6,110.0,22,1,4,4.0,28.0,1,128,2,1282
7,120.0,32,2,1,4.0,28.2,2,125,1,1251
9,178.0,56,1,4,3.0,33.6,2,126,2,1262
10,144.0,46,1,3,5.0,27.6,1,121,1,1211


## Intraclass correlation:


We can assess ICC using two regression techniques, marginal regression, and multilevel regression. We will start by using a technique called "Generalized Estimating Equations" (GEE) to fit marginal linear models, and to estimate the ICC for the NHANES clusters.

We will first look at the ICC for systolic blood pressure:

In [3]:
Nhanes_ds["smq"] = Nhanes_ds.SMQ020.replace({2: 0, 7: np.nan, 9: np.nan})

for item in ["BPXSY1", "RIDAGEYR", "BMXBMI", "smq", "SDMVSTRA"]:
    model = sm.GEE.from_formula(item + " ~ 1", groups="Group",
           cov_struct=sm.cov_struct.Exchangeable(), data=Nhanes_ds)
    result = model.fit()
    print(item, result.cov_struct.summary())

BPXSY1 The correlation between two observations in the same cluster is 0.030
RIDAGEYR The correlation between two observations in the same cluster is 0.035
BMXBMI The correlation between two observations in the same cluster is 0.039
smq The correlation between two observations in the same cluster is 0.026
SDMVSTRA The correlation between two observations in the same cluster is 0.959


Since Nhanes Data is collected using comples clustering method, we are trying to find out whether there is some correlation in the data coming from the same cluster.

SDMVSTRA has significantly higher correlation because it is the variable defining groups. 


## Conditional intraclass correlation
The ICC's studied above were marginal, in the sense that we were looking at whether, say, the SBP values were more similar within versus between clusters. To the extent that such "cluster effects" are found, it may be largely explained by demographic differences among the clusters. For example, we know from our previous analyses with the NHANES data that older people have higher SBP than younger people. Also, some clusters may contain a slightly older or younger set of people than others. Thus, by controlling for age, we might anticipate that the ICC will become smaller. This is shown in the next analysis:

In [4]:
model = sm.GEE.from_formula("BPXSY1 ~ RIDAGEYR " , groups = "Group", 
                           data = Nhanes_ds  , cov_struct = sm.cov_struct.Exchangeable())
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.019


The ICC drops from 0.03 to 0.019 buy controlling the age variable. 

## Marginal Linear Models 


In [5]:
Nhanes_ds["RIAGENDRx"] = Nhanes_ds.RIAGENDR.replace({1 : "Male" , 2 : "Female"})
# fit a linear model with OLS
model1 = sm.OLS.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)", data = Nhanes_ds)
result1 = model1.fit()

model2 = sm.GEE.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)" , groups = "Group",
                            cov_struct = sm.cov_struct.Exchangeable() , data = Nhanes_ds)
result2 = model2.fit()

x = pd.DataFrame({"OLS_params": result1.params , "OLS_SE" : result1.bse,
                 "GEE_params": result2.params, "GEE_SE": result2.bse})
x



,OLS_params,OLS_SE,GEE_params,GEE_SE
Intercept,91.736583,1.339378,92.168530,1.384309
RIAGENDRx[T.Male],3.671294,0.453763,3.650245,0.454498
C(RIDRETH1)[T.2],0.855488,0.819486,0.159296,0.767025
C(RIDRETH1)[T.3],-1.796132,0.671954,-2.233280,0.760228
C(RIDRETH1)[T.4],3.813314,0.732355,3.105654,0.881580
C(RIDRETH1)[T.5],-0.455347,0.808948,-0.439831,0.813675
RIDAGEYR,0.478699,0.012901,0.474101,0.018493
BMXBMI,0.278015,0.033285,0.280205,0.038553


As we can see the point estimates are nearly similar in OLS and marginal model but the variance is larger in GEE beacuse now we are also accounting for the dependence in data because of clustering effect. Marginal models are more flexible than OLS model.

## Marginal Logistic Model

In [6]:
Nhanes_ds["DMDEDUC2x"] = Nhanes_ds.DMDEDUC2.replace({1: "lt9", 2: "x9_11", 3: "HS", 4: "SomeCollege",
                                       5: "College", 7: np.nan, 9: np.nan})

# fit a basic GLM

model1 = sm.GLM.from_formula("smq ~ RIDAGEYR + RIAGENDRx + C(DMDEDUC2x)" , data = Nhanes_ds,
                            family = sm.families.Binomial())
result1 = model1.fit()

# fit a marginal GLM model with GEE

model2 = sm.GEE.from_formula("smq ~ RIDAGEYR + RIAGENDRx + C(DMDEDUC2x)" , data = Nhanes_ds,
                            family = sm.families.Binomial() , cov_struct = sm.cov_struct.Exchangeable(),
                            groups = "Group")
result2 = model2.fit()

x = pd.DataFrame({"GLM_params": result1.params , "GLM_SE" : result1.bse,
                 "GEE_params": result2.params, "GEE_SE": result2.bse})
x


,GLM_params,GLM_SE,GEE_params,GEE_SE
Intercept,-2.305999,0.114308,-2.249820,0.140567
RIAGENDRx[T.Male],0.909597,0.060167,0.908682,0.062342
C(DMDEDUC2x)[T.HS],0.943364,0.089663,0.887965,0.095397
C(DMDEDUC2x)[T.SomeCollege],0.832227,0.084361,0.771636,0.104449
C(DMDEDUC2x)[T.lt9],0.266228,0.109183,0.321784,0.141327
C(DMDEDUC2x)[T.x9_11],1.098561,0.106697,1.062149,0.138401
RIDAGEYR,0.018257,0.001725,0.017416,0.001803


As expected, the results show that the GLM and the GEE give very similar estimates for the regression parameters. However the standard errors obtained using GEE are somewhat larger than those obtained using GLM. This indicates that GLM understates the uncertainty in the estimated mean structure, which is a direct consequence of it ignoring the dependence structure. The GEE results do not suffer from this weakness.

## Multi Level Model

In [7]:
model = sm.MixedLM.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)", groups = "Group",
                               data = Nhanes_ds)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
           Mixed Linear Model Regression Results
============================================================
Model:             MixedLM  Dependent Variable:  BPXSY1     
No. Observations:  5102     Method:              REML       
No. Groups:        30       Scale:               256.6952   
Min. group size:   106      Log-Likelihood:      -21409.8702
Max. group size:   226      Converged:           Yes        
Mean group size:   170.1                                    
------------------------------------------------------------
                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------
Intercept         92.173    1.402 65.752 0.000 89.426 94.921
RIAGENDRx[T.Male]  3.650    0.452  8.084 0.000  2.765  4.535
C(RIDRETH1)[T.2]   0.153    0.887  0.172 0.863 -1.586  1.891
C(RIDRETH1)[T.3]  -2.238    0.758 -2.954 0.003 -3.723 -0.753
C(RIDRETH1)[T.4]   3.098    0.836  3.707 0.000  1.460  4.737
C(RIDRETH1)[T.5]  -0.439    0.878 -0.500 0.617 -2.161  1.282
RIDAGEYR           0.474    0.013 36.482 0.000  0.449  0.500
BMXBMI             0.280    0.033  8.404 0.000  0.215  0.346
Group Var          3.615    0.085                           
============================================================

"""

In [8]:
x = pd.DataFrame({"MixedLM_params": result.params , "MixedLM_SE" : result.bse,
                 "GEE_params": result2.params, "GEE_SE": result2.bse})
x

,MixedLM_params,MixedLM_SE,GEE_params,GEE_SE
BMXBMI,0.280217,0.033343,NaN,NaN
C(DMDEDUC2x)[T.HS],NaN,NaN,0.887965,0.095397
C(DMDEDUC2x)[T.SomeCollege],NaN,NaN,0.771636,0.104449
C(DMDEDUC2x)[T.lt9],NaN,NaN,0.321784,0.141327
C(DMDEDUC2x)[T.x9_11],NaN,NaN,1.062149,0.138401
C(RIDRETH1)[T.2],0.152885,0.887019,NaN,NaN
C(RIDRETH1)[T.3],-2.238069,0.757740,NaN,NaN
C(RIDRETH1)[T.4],3.098365,0.835925,NaN,NaN
C(RIDRETH1)[T.5],-0.439303,0.878201,NaN,NaN
Group Var,0.014084,0.005325,NaN,NaN


## Predicted random effects
While the actual random effects in a multilevel model are never observable, we can predict them from the data. This is sometimes useful, although the emphasis in multilevel regression usually lies with the structural parameters that underlie the random effects, and not the random effects themselves. In the NHANES analysis, for example, we could use the predicted random effects to quantify the uniqueness of each county relative to the mean.

The predicted random effects for the 30 groups in this analysis are shown below:

In [9]:
result.random_effects

{1191: Group   -1.630976
 dtype: float64,
 1192: Group   -0.086162
 dtype: float64,
 1201: Group   -2.042661
 dtype: float64,
 1202: Group   -0.147472
 dtype: float64,
 1211: Group    0.280623
 dtype: float64,
 1212: Group    1.580732
 dtype: float64,
 1221: Group    0.283347
 dtype: float64,
 1222: Group    0.131512
 dtype: float64,
 1231: Group   -2.038171
 dtype: float64,
 1232: Group    0.617651
 dtype: float64,
 1241: Group    2.878488
 dtype: float64,
 1242: Group   -0.519364
 dtype: float64,
 1251: Group    2.064967
 dtype: float64,
 1252: Group    1.521281
 dtype: float64,
 1261: Group   -1.261975
 dtype: float64,
 1262: Group    0.980846
 dtype: float64,
 1271: Group    0.118031
 dtype: float64,
 1272: Group   -0.128397
 dtype: float64,
 1281: Group   -0.384862
 dtype: float64,
 1282: Group   -3.582111
 dtype: float64,
 1291: Group   -3.271017
 dtype: float64,
 1292: Group   -0.829538
 dtype: float64,
 1301: Group   -0.884171
 dtype: float64,
 1302: Group    2.790657
 dtype: f

Based on these predicted random effects (also known as BLUPs), we see, for example, that cluster 1241 has unusually high SBP, and cluster 1282 has unusually low SBP. These deviations from what is expected are observed after adjusting for the covariates in the model, and hence are presumably driven by other characteristics of these clusters that are not reflected in the covariates.

## Random slopes

since we have found some correlation in the clusters. Next we will try to find out how the slope of one covariate varies within the clusters in each cluster. 
we will fit a random slopes model in which SBP will have a cluster specific intercept and slope for the age covariate.


In [23]:
Nhanes_ds["age_cen"] = Nhanes_ds.groupby("Group").RIDAGEYR.transform(lambda x : x - x.mean())
model = sm.MixedLM.from_formula("BPXSY1 ~ age_cen + RIAGENDRx + BMXBMI + C(RIDRETH1)" ,  data = Nhanes_ds,
                               groups = "Group" , vc_formula= {"age_cen" : "0 + age_cen"})
result = model.fit()
result.summary()



c:\users\prash\documents\ml\venv\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
             Mixed Linear Model Regression Results
===============================================================
Model:              MixedLM   Dependent Variable:   BPXSY1     
No. Observations:   5102      Method:               REML       
No. Groups:         30        Scale:                263.7323   
Min. group size:    106       Log-Likelihood:       -21469.9240
Max. group size:    226       Converged:            Yes        
Mean group size:    170.1                                      
---------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
---------------------------------------------------------------
Intercept         115.207    1.209 95.265 0.000 112.836 117.577
RIAGENDRx[T.Male]   3.643    0.457  7.962 0.000   2.746   4.539
C(RIDRETH1)[T.2]    1.167    0.827  1.412 0.158  -0.453   2.787
C(RIDRETH1)[T.3]   -1.659    0.679 -2.444 0.015  -2.989  -0.328
C(RIDRETH1)[T.4]    3.610    0.739  4.884 0.000   2.161   5.058
C(RIDRETH1)[T.5]   -1.208    0.816 -1.480 0.139  -2.807   0.392
age_cen             0.467    0.018 26.235 0.000   0.432   0.502
BMXBMI              0.288    0.034  8.574 0.000   0.222   0.353
age_cen Var         0.004    0.000                             
===============================================================

"""

The estimated variance for random age slopes  is 0.004, which translates to SD of 0.06. That is , from one cluster to another the age slope flactuates by $\pm 0.06-0.12$ standard deviations.These cluster-specific fluctuations are added/subtracted from the fixed effect for age, which is 0.467. Thus, in some clusters SBP may increase by around 0.467 + 0.06 = 0.527 mm/Hg per year, while in other clusters SBP may increase by only around 0.467 - 0.06 = 0.407 mm/Hg per year.

In [28]:
model = sm.MixedLM.from_formula("BPXSY1 ~ age_cen + RIAGENDRx + BMXBMI + C(RIDRETH1)" ,  data = Nhanes_ds,
                               groups = "Group" , re_formula = "1 + age_cen ")
result = model.fit()
result.summary()

c:\users\prash\documents\ml\venv\lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
              Mixed Linear Model Regression Results
=================================================================
Model:                MixedLM   Dependent Variable:   BPXSY1     
No. Observations:     5102      Method:               REML       
No. Groups:           30        Scale:                255.4451   
Min. group size:      106       Log-Likelihood:       -21413.6193
Max. group size:      226       Converged:            Yes        
Mean group size:      170.1                                      
-----------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
-----------------------------------------------------------------
Intercept           115.467    1.340 86.173 0.000 112.840 118.093
RIAGENDRx[T.Male]     3.662    0.451  8.121 0.000   2.778   4.546
C(RIDRETH1)[T.2]      0.023    0.898  0.025 0.980  -1.738   1.783
C(RIDRETH1)[T.3]     -2.251    0.778 -2.893 0.004  -3.775  -0.726
C(RIDRETH1)[T.4]      3.011    0.854  3.524 0.000   1.336   4.686
C(RIDRETH1)[T.5]     -0.585    0.893 -0.655 0.512  -2.336   1.165
age_cen               0.466    0.018 26.286 0.000   0.431   0.501
BMXBMI                0.283    0.033  8.497 0.000   0.218   0.349
Group Var             8.655    0.169                             
Group x age_cen Cov   0.119    0.004                             
age_cen Var           0.004    0.000                             
=================================================================

"""

The estimated correlation coefficient between random slopes and random
intercepts is estimated to be `0.119/sqrt(8.655*0.004)`, which is
around `0.64`.  This indicates that the clusters with unusually high
average SBP also tend to have SBP increasing faster with age.  Note
however that these structural parameters are only estimates, and due
to the variance parameter falling close to the boundary, the estimates
may not be particularly precise.
